# Intersect mutations with genomic elements

In [1]:
import gzip
from collections import defaultdict

from intervaltree import IntervalTree
import pandas as pd

In [2]:
sig_assignments = "sig_assignments.txt"
sig_assignments_df = pd.read_csv(sig_assignments, sep='\t', header=0)
sig_assignments_df = sig_assignments_df.drop(sig_assignments_df.columns[0], axis=1)
sig_assignments_df.head()

,chr,pos,ref,alt,sample,mutation_type,SBS1,SBS5,SBS17a,SBS17b,SBS18,SBS31,SBS35,max_sig
0,chr1,1541060,C,T,5FU-PATIENT1-N-CLONE1,A[C>T]C,0.185835,0.649494,0.0,0.0,0.070642,0.056216,0.037813,SBS5
1,chr1,1937385,C,A,5FU-PATIENT1-N-CLONE1,T[C>A]T,0.000070,0.185320,0.0,0.0,0.745359,0.010938,0.058313,SBS18
2,chr1,2612326,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]G,0.856552,0.094373,0.0,0.0,0.037932,0.007383,0.003761,SBS1
3,chr1,3510207,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]A,0.047488,0.568417,0.0,0.0,0.143529,0.201537,0.039029,SBS5
4,chr1,3510227,C,A,5FU-PATIENT1-N-CLONE1,G[C>A]C,0.008619,0.240045,0.0,0.0,0.227282,0.112632,0.411422,SBS35


In [3]:
#remove all rows where max_sig != SBS31 or SBS35

sigs_of_interest = sig_assignments_df[sig_assignments_df['max_sig'].isin(['SBS31', 'SBS35'])]
sigs_of_interest.head()

,chr,pos,ref,alt,sample,mutation_type,SBS1,SBS5,SBS17a,SBS17b,SBS18,SBS31,SBS35,max_sig
4,chr1,3510227,C,A,5FU-PATIENT1-N-CLONE1,G[C>A]C,8.618724e-03,0.240045,0.0,0.0,0.227282,0.112632,0.411422,SBS35
9,chr1,4694445,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]T,2.539806e-03,0.369441,0.0,0.0,0.044529,0.387162,0.196328,SBS31
10,chr1,4845854,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]T,2.539806e-03,0.369441,0.0,0.0,0.044529,0.387162,0.196328,SBS31
17,chr1,7428099,C,A,5FU-PATIENT1-N-CLONE1,C[C>A]C,5.504499e-02,0.232994,0.0,0.0,0.311401,0.073490,0.327071,SBS35
21,chr1,10829656,C,A,5FU-PATIENT1-N-CLONE1,C[C>A]T,3.772539e-15,0.135507,0.0,0.0,0.317330,0.097255,0.449907,SBS35


In [4]:
len(sigs_of_interest)

9675

In [5]:
#patients present in sigs_of_interest

for sample in sigs_of_interest:
    print(sigs_of_interest[sample].unique())

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7' 'chr8' 'chr9' 'chr10'
 'chr11' 'chr12' 'chr13' 'chr14' 'chr15' 'chr16' 'chr17' 'chr18' 'chr19'
 'chr20' 'chr21' 'chr22' 'chrX' 'chrY']
[ 3510227  4694445  4845854 ... 25831858 58532378 74901909]
['C' 'T']
['A' 'T' 'G']
['5FU-PATIENT1-N-CLONE1' '5FU-PATIENT1-N-CLONE2'
 '5FU-PATIENT10-LIVN-CLONE1' '5FU-PATIENT10-LIVN-CLONE4'
 '5FU-PATIENT11-LIVN-CLONE4' '5FU-PATIENT12-COLN-CLONE4'
 '5FU-PATIENT13-COLN-CLONE1' '5FU-PATIENT13-COLN-CLONE2'
 '5FU-PATIENT14-LIVN-CLONE1' '5FU-PATIENT2-N-CLONE1'
 '5FU-PATIENT2-N-CLONE2' '5FU-PATIENT2-N-CLONE3' '5FU-PATIENT2-N-CLONE6'
 '5FU-PATIENT3-N-CLONE4' '5FU-PATIENT4-N-CLONE1' '5FU-PATIENT4-N-CLONE2'
 '5FU-PATIENT5-N-CLONE2' '5FU-PATIENT5-N-CLONE3'
 '5FU-PATIENT6-LivN-CLONE2' '5FU-PATIENT6-LivN-CLONE5'
 '5FU-PATIENT7-LivN-CLONE1' '5FU-PATIENT7-LivN-CLONE4'
 '5FU-PATIENT8-LivN-CLONE13' '5FU-PATIENT8-LivN-CLONE15'
 '5FU-PATIENT9-LivN-CLONE3' 'c080216DCLONE3' 'c110116DCLONE2'
 'C30913DClone12' 'C30913DClone13'

In [33]:
#counting number of mutations per patient
sample = "C30913DClone13"

only_SBS31 = sigs_of_interest[sigs_of_interest['max_sig'].isin(['SBS31'])]
only_SBS35 = sigs_of_interest[sigs_of_interest['max_sig'].isin(['SBS35'])]

print(sum(only_SBS31["sample"] == sample))
print(sum(only_SBS35["sample"] == sample))
print(sum(sigs_of_interest["sample"] == sample))

0
25
25


In [7]:
# Load elements into intervaltree
elements_f = 'hg19_cds.transcript_level.overlap.gz'
tree = defaultdict(IntervalTree)
with gzip.open(elements_f, 'rt') as fd:
    next(fd)    # Skip header
    for line in fd:
        chrom, start, end, strand, gene_id, transcript_id, symbol = line.strip().split('\t')
        tree[chrom].addi(int(start), int(end) + 1, f'{symbol}_{transcript_id}')    # +1 open end


In [7]:
tree["1"]

IntervalTree([Interval(65565, 65574, 'OR4F5_ENST00000641515'), Interval(69037, 70006, 'OR4F5_ENST00000641515'), Interval(69091, 70006, 'OR4F5_ENST00000335137'), Interval(138533, 139310, 'AL627309.1_ENST00000423372'), Interval(367659, 368595, 'OR4F29_ENST00000426406'), Interval(621099, 622035, 'OR4F16_ENST00000332831'), Interval(859812, 860329, 'SAMD11_ENST00000420190'), Interval(861302, 861394, 'SAMD11_ENST00000420190'), Interval(861322, 861394, 'SAMD11_ENST00000342066'), Interval(861322, 861394, 'SAMD11_ENST00000437963'), Interval(861322, 861394, 'SAMD11_ENST00000616016'), Interval(861322, 861394, 'SAMD11_ENST00000616125'), Interval(861322, 861394, 'SAMD11_ENST00000617307'), Interval(861322, 861394, 'SAMD11_ENST00000618181'), Interval(861322, 861394, 'SAMD11_ENST00000618323'), Interval(861322, 861394, 'SAMD11_ENST00000618779'), Interval(861322, 861394, 'SAMD11_ENST00000620200'), Interval(861322, 861394, 'SAMD11_ENST00000622503'), Interval(865535, 865717, 'SAMD11_ENST00000342066'), Int

In [8]:
# Read mutations attributed to SBS31 and SBS35
# For each mutation...

# Intersect the tree as follows (this step is not needed; just do the next one)
chrom = '1'
position = 65570
print(tree[chrom][position])

# Retrieve the gene symbol and transcript ID of the intersection (if any) like this: 
for interval in tree[chrom][position]: 
    gene_transid = interval.data
    print(gene_transid)

{Interval(65565, 65574, 'OR4F5_ENST00000641515')}
OR4F5_ENST00000641515


In [9]:
sigs_of_interest.head()

,chr,pos,ref,alt,sample,mutation_type,SBS1,SBS5,SBS17a,SBS17b,SBS18,SBS31,SBS35,max_sig
4,chr1,3510227,C,A,5FU-PATIENT1-N-CLONE1,G[C>A]C,8.618724e-03,0.240045,0.0,0.0,0.227282,0.112632,0.411422,SBS35
9,chr1,4694445,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]T,2.539806e-03,0.369441,0.0,0.0,0.044529,0.387162,0.196328,SBS31
10,chr1,4845854,C,T,5FU-PATIENT1-N-CLONE1,C[C>T]T,2.539806e-03,0.369441,0.0,0.0,0.044529,0.387162,0.196328,SBS31
17,chr1,7428099,C,A,5FU-PATIENT1-N-CLONE1,C[C>A]C,5.504499e-02,0.232994,0.0,0.0,0.311401,0.073490,0.327071,SBS35
21,chr1,10829656,C,A,5FU-PATIENT1-N-CLONE1,C[C>A]T,3.772539e-15,0.135507,0.0,0.0,0.317330,0.097255,0.449907,SBS35


In [12]:
#creating a list containing all the genomic element information

genomic_elements = []
for _, row in sigs_of_interest.iterrows():
    chrom = row['chr']
    chrom = chrom.removeprefix('chr')
    position = row['pos']
    sample = row['sample']
    mutation = row['mutation_type']
    transcript_list = []

    for interval in tree[chrom][position]: 
        gene_transid = interval.data
        transcript_list += [gene_transid]
        
        #to find mutations that overlap PCBP1
        if "SLC35A3" in gene_transid: 
            print(mutation)
            print(chrom)
            print(position)
            print(sample)
            
    genomic_elements += [';;'.join(transcript_list)]

    
    
        
#print(genomic_elements) #this is just a check

C[C>A]T
1
100492748
5FU-PATIENT4-N-CLONE2


In [11]:
list_of_transcripts = list(filter(None, genomic_elements))
print(list_of_transcripts)
len(list_of_transcripts)

['TMEM69_ENST00000372025', 'GSTCD_ENST00000507281;;GSTCD_ENST00000515279;;GSTCD_ENST00000360505;;GSTCD_ENST00000394730;;GSTCD_ENST00000394728', 'RANBP3L_ENST00000296604;;RANBP3L_ENST00000502994;;RANBP3L_ENST00000515759', 'MRPS27_ENST00000261413;;MRPS27_ENST00000515404;;MRPS27_ENST00000508863;;MRPS27_ENST00000513900;;MRPS27_ENST00000457646', 'EEF1D_ENST00000531281;;EEF1D_ENST00000525261;;EEF1D_ENST00000526710;;EEF1D_ENST00000534804;;EEF1D_ENST00000530306;;EEF1D_ENST00000524900;;EEF1D_ENST00000532596;;EEF1D_ENST00000532741;;EEF1D_ENST00000528519;;EEF1D_ENST00000524883;;EEF1D_ENST00000442189;;EEF1D_ENST00000423316;;EEF1D_ENST00000529832;;EEF1D_ENST00000531670;;EEF1D_ENST00000530545;;EEF1D_ENST00000526135;;EEF1D_ENST00000618139;;EEF1D_ENST00000531953', 'CFAP54_ENST00000524981;;CFAP54_ENST00000637336', 'ACOT1_ENST00000311148;;ACOT1_ENST00000557556', 'LTBP2_ENST00000261978;;LTBP2_ENST00000556690', 'TEX2_ENST00000584379;;TEX2_ENST00000583097;;TEX2_ENST00000615733;;TEX2_ENST00000258991', 'LCE3

86

In [12]:
final_transcripts = []
for i in list_of_transcripts:
    temp_list = i.split(';;')
    for j in temp_list:
        final_transcripts.append(j)
print(final_transcripts)
len(final_transcripts)

['TMEM69_ENST00000372025', 'GSTCD_ENST00000507281', 'GSTCD_ENST00000515279', 'GSTCD_ENST00000360505', 'GSTCD_ENST00000394730', 'GSTCD_ENST00000394728', 'RANBP3L_ENST00000296604', 'RANBP3L_ENST00000502994', 'RANBP3L_ENST00000515759', 'MRPS27_ENST00000261413', 'MRPS27_ENST00000515404', 'MRPS27_ENST00000508863', 'MRPS27_ENST00000513900', 'MRPS27_ENST00000457646', 'EEF1D_ENST00000531281', 'EEF1D_ENST00000525261', 'EEF1D_ENST00000526710', 'EEF1D_ENST00000534804', 'EEF1D_ENST00000530306', 'EEF1D_ENST00000524900', 'EEF1D_ENST00000532596', 'EEF1D_ENST00000532741', 'EEF1D_ENST00000528519', 'EEF1D_ENST00000524883', 'EEF1D_ENST00000442189', 'EEF1D_ENST00000423316', 'EEF1D_ENST00000529832', 'EEF1D_ENST00000531670', 'EEF1D_ENST00000530545', 'EEF1D_ENST00000526135', 'EEF1D_ENST00000618139', 'EEF1D_ENST00000531953', 'CFAP54_ENST00000524981', 'CFAP54_ENST00000637336', 'ACOT1_ENST00000311148', 'ACOT1_ENST00000557556', 'LTBP2_ENST00000261978', 'LTBP2_ENST00000556690', 'TEX2_ENST00000584379', 'TEX2_ENST0

293

In [13]:
#write list of transcripts to empty txt file

# open file in write mode
with open(r'list_of_transcripts.txt', 'w') as fp:
    for item in final_transcripts:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [14]:
#create list of transcripts without ENST

transcripts_2 = []
for i in final_transcripts:
    transcripts_2.append(i.split('_')[0])
print(transcripts_2)
len(transcripts_2)

['TMEM69', 'GSTCD', 'GSTCD', 'GSTCD', 'GSTCD', 'GSTCD', 'RANBP3L', 'RANBP3L', 'RANBP3L', 'MRPS27', 'MRPS27', 'MRPS27', 'MRPS27', 'MRPS27', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'EEF1D', 'CFAP54', 'CFAP54', 'ACOT1', 'ACOT1', 'LTBP2', 'LTBP2', 'TEX2', 'TEX2', 'TEX2', 'TEX2', 'LCE3B', 'TMF1', 'TMF1', 'TMF1', 'MDN1', 'MDN1', 'MDN1', 'CCL24', 'CCL24', 'INPP5F', 'INPP5F', 'INPP5F', 'INPP5F', 'KRT84', 'KRT5', 'KRT5', 'PIGN', 'PIGN', 'PIGN', 'FCHO1', 'FCHO1', 'FCHO1', 'FCHO1', 'FCHO1', 'FCHO1', 'FCHO1', 'GATA1', 'GATA1', 'PCBP1', 'KRTAP19-1', 'PEF1', 'ADAMTS4', 'ACTN2', 'ACTN2', 'ACTN2', 'ACTN2', 'TRUB2', 'MUC2', 'TSC22D1', 'DUXA', 'CDH26', 'ZCCHC17', 'PDE3A', 'BEGAIN', 'UNC45B', 'UNC45B', 'UNC45B', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'KXD1', 'USP51', 'TMEM164', 'TMEM164', 'TMEM164', 'RIMS1', 'RIMS1', 'CAPRIN2', 'CAPRIN2', 'CAPRI

293

In [15]:
#write new list of transcripts to empty txt file (check file first!)

# open file in write mode
with open(r'transcripts_2.txt', 'w') as fp:
    for item in transcripts_2:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [62]:
#count unique transcripts

import numpy as np

values, counts = np.unique(transcripts_2, return_counts=True)

print(values, counts)
len(values)

NameError: name 'transcripts_2' is not defined